In [2]:
p=2^372*3^239-1 
K.<a>=GF(p^2,'a',modulus=[1,0,1])
root=a
K=GF(p^2,'a',modulus=[1,0,1])
E=EllipticCurve(K,[1,0])
n=372
C=E(0,0)
Infi=(2*C)
def distorsion_map(P):
    x=-P[0]
    y=P[1]*root
    return E(x,y)
# generates 3 torsion
P=E(6,4688558410880967105802548485745508960029362498156488028653160515710832445592615410221556012727376816077230428365690034126899246042233876329216033729810811325868800516984641936994142700675815111949425502583260986657458474705646)
P_A=(2^372)*P
Q_A=distorsion_map(P_A)
# generates 2 torsion
P2=E(11,4178137817326599126129602268030084854254919541457752920496919677484559856346888397586423437889634773403442208976556528418523132894243487824255334881927098101022506929946230877638251483704236124248972585701878366089218056349632)
P_B=(3^239)*P2
Q_B=distorsion_map(P_B)
def three_point_ladder(P,Q,alpha):
    if(alpha==0):
        return P
    else:
        P_Q=(P-Q)[0]
        digit_store=[]
        while(alpha):
            digit_store.append(alpha&1)
            alpha=alpha//2
        digit_store.reverse()
        flag=0
        A_X=0
        A_Z=0
        B_X=Q[0]
        B_Z=1
        C_X=P[0]
        C_Z=1
        while (len(digit_store)!=0):
            c=digit_store[0]
            temp_A_X=A_X
            temp_A_Z=A_Z
            temp_B_X=B_X
            temp_B_Z=B_Z
            temp_C_X=C_X
            temp_C_Z=C_Z
            if(c&1==1 and flag==0):
                A_X=Q[0]
                A_Z=1
                # DOUBLE B
                B_X,B_Z=(temp_B_X^2-temp_B_Z^2)^2,4*temp_B_X*temp_B_Z*(temp_B_X^2+temp_B_Z^2)
                # ladder on C=dadd(B,C,Q-P)
                C_X,C_Z=(temp_B_X*temp_C_X-temp_B_Z*temp_C_Z)^2,P_Q*(temp_B_X*temp_C_Z-temp_B_Z*temp_C_X)^2
                flag=1
            elif(c&1==0 and flag==1):
                # DOUBLE A
                A_X,A_Z=(temp_A_X^2-temp_A_Z^2)^2,4*temp_A_X*temp_A_Z*(temp_A_X^2+temp_A_Z^2)
                # ladder on B=dadd(A,B,Q)
                B_X,B_Z=(temp_A_X*temp_B_X-temp_A_Z*temp_B_Z)^2,Q[0]*(temp_A_X*temp_B_Z-temp_A_Z*temp_B_X)^2
                # ladder on C=dadd(A,C,P)
                C_X,C_Z=(temp_A_X*temp_C_X-temp_A_Z*temp_C_Z)^2,P[0]*(temp_A_X*temp_C_Z-temp_A_Z*temp_C_X)^2
            elif(c&1==1 and flag==1):
                # ladder on A=dadd(A,B,Q)
                A_X,A_Z=(temp_A_X*temp_B_X-temp_A_Z*temp_B_Z)^2,Q[0]*(temp_A_X*temp_B_Z-temp_A_Z*temp_B_X)^2
                # DOUBLE B
                B_X,B_Z=(temp_B_X^2-temp_B_Z^2)^2,4*temp_B_X*temp_B_Z*(temp_B_X^2+temp_B_Z^2)
                # ladder on C=dadd(B,C,Q-P)
                C_X,C_Z=(temp_B_X*temp_C_X-temp_B_Z*temp_C_Z)^2,P_Q*((temp_B_X*temp_C_Z)-(temp_B_Z*temp_C_X))^2
            elif(c&1==0 and flag==0):
                continue
            digit_store.pop(0)
        return((C_X*(C_Z^-1)),(A_X*(A_Z^-1)))
def okeya_sakurai(P,Q,alpha):
    x=P[0]
    y=P[1]
    result=three_point_ladder(P,Q,alpha)
    x2=result[0]
    x1=result[1]
    y2=((x+x2)*((x*x2)+1)-((x2-x)^2)*x1)*((2*y)^-1)
    return E(x2,-y2)
%timeit(okeya_sakurai(P_A,Q_A,546546543213453121354635132132354351))
%timeit(P_A+546546543213453121354635132132354351*Q_A)
print(okeya_sakurai(P_A,Q_A,546546543213453121354635132132354351))
print(P_A+546546543213453121354635132132354351*Q_A)

100 loops, best of 3: 15.3 ms per loop
10 loops, best of 3: 28.7 ms per loop
(5863717474777048101810207934857415030594706567964356052736942411195400493055053733187413534137158032719195417532014188709100142716090271117452149177159814355752733478013748579530493639694860794125663295094847603034422835665712*a + 1161746865679931320918627297028150730954488808930177027012667101582469528032818073514128530910289366449926617446718859783853568400246815753669281186529405886583694675626985329784887651579461184251300796867742974017972017229713 : 7147563121740873491592538239082072063978689737920023775292018533203296579663538123460211182199516221687058083792212298949022711456567727908007803919567744965656531316962478198541625336363257655561295218264650765820567671430161*a + 1041358648835737042676902367953666151728042749642074409097350819992478034904032125592061445596184667528404200957379783605002521359167921251736897576014874985486760941233124193699749730478455708425680721855183404172678147274003 : 1)